In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pickle

In [2]:
df = pd.read_csv('new_heart.csv', sep=',', encoding='utf-8')

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0


In [4]:
#Transformando Valores Nominais em Ordinais

df_alterado = df.iloc[:, 0:12].values
lista = [1,2,6,8,10]
for i in lista:
    df_alterado[:,i] = LabelEncoder().fit_transform(df_alterado[:,i])

#Transformando em DataFrame novamente

colunas = df.columns[0:12]

df_alterado = pd.DataFrame(data=df_alterado, columns=colunas)

for i in range(12):
    df_alterado[df_alterado.columns[i]] = df_alterado[df_alterado.columns[i]].astype(int)

df_alterado

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0,2,0
1,49,0,2,160,180,0,1,156,0,1,1,1
2,37,1,1,130,283,0,2,98,0,0,2,0
3,48,0,0,138,214,0,1,108,1,1,1,1
4,54,1,2,150,195,0,1,122,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
912,45,1,3,110,264,0,1,132,0,1,1,1
913,68,1,0,144,193,1,1,141,0,3,1,1
914,57,1,0,130,131,0,1,115,1,1,1,1
915,57,0,1,130,236,0,0,174,0,0,1,1


In [5]:
previsores = df_alterado.iloc[:, 0:11].values
previsores

array([[40,  1,  1, ...,  0,  0,  2],
       [49,  0,  2, ...,  0,  1,  1],
       [37,  1,  1, ...,  0,  0,  2],
       ...,
       [57,  1,  0, ...,  1,  1,  1],
       [57,  0,  1, ...,  0,  0,  1],
       [38,  1,  2, ...,  0,  0,  2]])

In [6]:
# Armazenando coluna HeartDisease para testes
alvo = df_alterado.iloc[:, 11].values

In [7]:
df_alterado.describe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.00000,917.000000
mean,53.509269,0.789531,0.779716,132.540894,244.516903,0.233370,0.989095,136.789531,0.404580,0.719738,1.36205,0.552890
std,9.437636,0.407864,0.956193,17.999749,53.347700,0.423206,0.632016,25.467129,0.491078,0.990662,0.60727,0.497466
min,28.000000,0.000000,0.000000,80.000000,85.000000,0.000000,0.000000,60.000000,0.000000,-2.000000,0.00000,0.000000
25%,47.000000,1.000000,0.000000,120.000000,214.000000,0.000000,1.000000,120.000000,0.000000,0.000000,1.00000,0.000000
50%,54.000000,1.000000,0.000000,130.000000,244.000000,0.000000,1.000000,138.000000,0.000000,0.000000,1.00000,1.000000
75%,60.000000,1.000000,2.000000,140.000000,267.000000,0.000000,1.000000,156.000000,1.000000,1.000000,2.00000,1.000000
max,77.000000,1.000000,3.000000,200.000000,603.000000,1.000000,2.000000,202.000000,1.000000,6.000000,2.00000,1.000000


In [8]:
previsores_esc = StandardScaler().fit_transform(previsores)
previsores_esc

array([[-1.43220634,  0.51630861,  0.23050141, ..., -0.82431012,
        -0.72691898,  1.05109458],
       [-0.47805725, -1.9368261 ,  1.27688653, ..., -0.82431012,
         0.28305784, -0.59651863],
       [-1.75025603,  0.51630861,  0.23050141, ..., -0.82431012,
        -0.72691898,  1.05109458],
       ...,
       [ 0.37007527,  0.51630861, -0.8158837 , ...,  1.21313565,
         0.28305784, -0.59651863],
       [ 0.37007527, -1.9368261 ,  0.23050141, ..., -0.82431012,
        -0.72691898, -0.59651863],
       [-1.64423947,  0.51630861,  1.27688653, ..., -0.82431012,
        -0.72691898,  1.05109458]])

In [9]:
previsoresdf = pd.DataFrame(previsores_esc)
previsoresdf

,0,1,2,3,4,5,6,7,8,9,10
0,-1.432206,0.516309,0.230501,0.414627,0.834288,-0.551733,0.017264,1.383339,-0.824310,-0.726919,1.051095
1,-0.478057,-1.936826,1.276887,1.526360,-1.210026,-0.551733,0.017264,0.754736,-0.824310,0.283058,-0.596519
2,-1.750256,0.516309,0.230501,-0.141240,0.721757,-0.551733,1.600366,-1.523953,-0.824310,-0.726919,1.051095
3,-0.584074,-1.936826,-0.815884,0.303453,-0.572350,-0.551733,0.017264,-1.131075,1.213136,0.283058,-0.596519
4,0.052026,0.516309,1.276887,0.970493,-0.928698,-0.551733,0.017264,-0.581047,-0.824310,-0.726919,1.051095
...,...,...,...,...,...,...,...,...,...,...,...
912,-0.902124,0.516309,2.323272,-1.252973,0.365409,-0.551733,0.017264,-0.188170,-0.824310,0.283058,-0.596519
913,1.536257,0.516309,-0.815884,0.636973,-0.966209,1.812470,0.017264,0.165420,-0.824310,2.303011,-0.596519
914,0.370075,0.516309,-0.815884,-0.141240,-2.129030,-0.551733,0.017264,-0.856061,1.213136,0.283058,-0.596519
915,0.370075,-1.936826,0.230501,-0.141240,-0.159736,-0.551733,-1.565838,1.461915,-0.824310,-0.726919,-0.596519


In [10]:
previsoresdf.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02
mean,1.859654e-16,-7.748558e-17,7.748558e-18,7.767929e-16,-2.247082e-16,4.649135e-17,-6.198846e-17,-5.114048e-16,-1.046055e-16,2.324567e-17,3.874279e-17
std,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00
min,-2.704405e+00,-1.936826e+00,-8.158837e-01,-2.920572e+00,-2.991768e+00,-5.517333e-01,-1.565838e+00,-3.016886e+00,-8.243101e-01,-2.746873e+00,-2.244132e+00
25%,-6.900904e-01,5.163086e-01,-8.158837e-01,-6.971063e-01,-5.723500e-01,-5.517333e-01,1.726392e-02,-6.596226e-01,-8.243101e-01,-7.269190e-01,-5.965186e-01
50%,5.202558e-02,5.163086e-01,-8.158837e-01,-1.412398e-01,-9.694608e-03,-5.517333e-01,1.726392e-02,4.755658e-02,-8.243101e-01,-7.269190e-01,-5.965186e-01
75%,6.881250e-01,5.163086e-01,1.276887e+00,4.146267e-01,4.216745e-01,-5.517333e-01,1.726392e-02,7.547357e-01,1.213136e+00,2.830578e-01,1.051095e+00
max,2.490407e+00,5.163086e-01,2.323272e+00,3.749826e+00,6.723415e+00,1.812470e+00,1.600366e+00,2.561971e+00,1.213136e+00,5.332942e+00,1.051095e+00


In [11]:
previsoresHot = ColumnTransformer(transformers=
                                  [('OneHot', OneHotEncoder(), [1,2,6,8,10])], remainder='passthrough').fit_transform(previsores)

In [12]:
previsoresHot

array([[  0.,   1.,   0., ...,   0., 172.,   0.],
       [  1.,   0.,   0., ...,   0., 156.,   1.],
       [  0.,   1.,   0., ...,   0.,  98.,   0.],
       ...,
       [  0.,   1.,   1., ...,   0., 115.,   1.],
       [  1.,   0.,   0., ...,   0., 174.,   0.],
       [  0.,   1.,   0., ...,   0., 173.,   0.]])

In [13]:
previsoresHot_esc = StandardScaler().fit_transform(previsoresHot)
previsoresHotdf = pd.DataFrame(previsoresHot_esc)
previsoresHotdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.516309,0.516309,-1.085425,2.073784,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,-1.001091,1.149573,-1.432206,0.414627,0.834288,-0.551733,1.383339,-0.726919
1,1.936826,-1.936826,-1.085425,-0.482210,1.881384,-0.229810,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,-0.478057,1.526360,-1.210026,-0.551733,0.754736,0.283058
2,-0.516309,0.516309,-1.085425,2.073784,-0.531524,-0.229810,-0.507826,-1.226974,2.037569,0.824310,-0.824310,-0.271607,-1.001091,1.149573,-1.750256,-0.141240,0.721757,-0.551733,-1.523953,-0.726919
3,1.936826,-1.936826,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,-1.213136,1.213136,-0.271607,0.998910,-0.869888,-0.584074,0.303453,-0.572350,-0.551733,-1.131075,0.283058
4,-0.516309,0.516309,-1.085425,-0.482210,1.881384,-0.229810,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,-1.001091,1.149573,0.052026,0.970493,-0.928698,-0.551733,-0.581047,-0.726919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,-0.516309,0.516309,-1.085425,-0.482210,-0.531524,4.351412,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,-0.902124,-1.252973,0.365409,-0.551733,-0.188170,0.283058
913,-0.516309,0.516309,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,1.536257,0.636973,-0.966209,1.812470,0.165420,2.303011
914,-0.516309,0.516309,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,-1.213136,1.213136,-0.271607,0.998910,-0.869888,0.370075,-0.141240,-2.129030,-0.551733,-0.856061,0.283058
915,1.936826,-1.936826,-1.085425,2.073784,-0.531524,-0.229810,1.969177,-1.226974,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,0.370075,-0.141240,-0.159736,-0.551733,1.461915,-0.726919


In [14]:
previsoresHotdf.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,917.000000,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,917.000000,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02
mean,1.084798e-16,-1.472226e-16,1.937139e-17,-3.874279e-17,3.874279e-17,6.973702e-17,0.000000,-9.298269e-17,1.549712e-17,-4.261707e-17,4.261707e-17,8.523413e-17,0.000000,-3.874279e-17,1.859654e-16,7.884157e-16,-2.276139e-16,-1.549712e-17,-5.114048e-16,3.099423e-17
std,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00
min,-5.163086e-01,-1.936826e+00,-1.085425e+00,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,-1.226974e+00,-4.907810e-01,-1.213136e+00,-8.243101e-01,-2.716072e-01,-1.001091,-8.698879e-01,-2.704405e+00,-2.920572e+00,-2.991768e+00,-5.517333e-01,-3.016886e+00,-2.746873e+00
25%,-5.163086e-01,5.163086e-01,-1.085425e+00,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,-1.226974e+00,-4.907810e-01,-1.213136e+00,-8.243101e-01,-2.716072e-01,-1.001091,-8.698879e-01,-6.900904e-01,-6.971063e-01,-5.723500e-01,-5.517333e-01,-6.596226e-01,-7.269190e-01
50%,-5.163086e-01,5.163086e-01,9.212982e-01,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,8.150134e-01,-4.907810e-01,8.243101e-01,-8.243101e-01,-2.716072e-01,0.998910,-8.698879e-01,5.202558e-02,-1.412398e-01,-9.694608e-03,-5.517333e-01,4.755658e-02,-7.269190e-01
75%,-5.163086e-01,5.163086e-01,9.212982e-01,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,8.150134e-01,-4.907810e-01,8.243101e-01,1.213136e+00,-2.716072e-01,0.998910,1.149573e+00,6.881250e-01,4.146267e-01,4.216745e-01,-5.517333e-01,7.547357e-01,2.830578e-01
max,1.936826e+00,5.163086e-01,9.212982e-01,2.073784e+00,1.881384e+00,4.351412e+00,1.969177,8.150134e-01,2.037569e+00,8.243101e-01,1.213136e+00,3.681787e+00,0.998910,1.149573e+00,2.490407e+00,3.749826e+00,6.723415e+00,1.812470e+00,2.561971e+00,5.332942e+00


### Info Variáveis

alvo = variável que se pretende atingir (tem ou não doença cardíaca)

previsores = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas manualmente, sem escalonar.

previsores_esc = previsores escalonado

previsoresHot = conjunto de varáveis previsoras tranformadas pelo labelencode e onehotencoder, sem escalonar

previsoresHot_esc = previsoresHot escalonado

### Salvando as Variáveis

In [39]:
arq1 = open('alvo.pkl', 'wb')
pickle.dump(alvo, arq1)
arq2 = open('previsores.pkl', 'wb')
pickle.dump(previsores, arq2)
arq3 = open('previsores_esc.pkl', 'wb')
pickle.dump(previsores_esc, arq3)
arq4 = open('previsoresHot.pkl', 'wb')
pickle.dump(previsoresHot, arq4)
arq5 = open('previsoresHot_esc.pkl', 'wb')
pickle.dump(previsoresHot_esc, arq5)
arq5.close()

In [40]:
arq2 = open('previsores.pkl', 'rb')
previsores = pickle.load(arq2)
previsores

array([[40,  1,  1, ...,  0,  0,  2],
       [49,  0,  2, ...,  0,  1,  1],
       [37,  1,  1, ...,  0,  0,  2],
       ...,
       [57,  1,  0, ...,  1,  1,  1],
       [57,  0,  1, ...,  0,  0,  1],
       [38,  1,  2, ...,  0,  0,  2]])